In [1]:
import findspark
findspark.init()
import shapely
import pandas as pd
import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from  pyspark.sql.functions import rand
from pyspark.sql.functions import lower,col,lit
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from  pyspark.sql.functions import rand
from pyspark.sql.functions import lower,col,lit

/opt/anaconda3/envs/bigdata/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
import requests #for querying inegis database

In [3]:
spark = SparkSession. \
builder. \
appName('appName'). \
config("spark.serializer", KryoSerializer.getName). \
config("spark.executor.memory", "5g"). \
config("spark.driver.memory", "10g"). \
config('spark.driver.maxResultSize', '5g'). \
config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.2.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
getOrCreate()
SedonaRegistrator.registerAll(spark)

22/11/28 14:47:54 WARN Utils: Your hostname, MacBook-Air-de-Antonio-3.local resolves to a loopback address: 127.0.0.1; using 10.22.135.81 instead (on interface en0)
22/11/28 14:47:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/anaconda3/envs/bigdata/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/antoniopatjane/.ivy2/cache
The jars for the packages stored in: /Users/antoniopatjane/.ivy2/jars
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bc43d7ff-71be-4a5a-b4cd-dc3b807d4ee9;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.2.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.2.0-incubating in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.5.0 in central
	found org.apache.sedona#sedona-sql-3.0_2.12;1.2.0-incubating in central
	found org.datasyslab#geotools-wrapper;1.1.0-25.2 in central
:: resolution report :: resolve 129ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.sedona#sedona-core-3.0_2.12;1.2.0-incubating from central i

22/11/28 14:47:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 14:47:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 14:47:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


True

In [4]:
BD_MANZANAS = spark.read.parquet(f"/Users/antoniopatjane/Downloads/19_NL.parquet")
#BD_MANZANAS.printSchema()

In [5]:
#list of the database selected variables
var_viv = ["CVEGEO", "VIV0", "VIV1", "VIV7", "VIV8", "VIV9", "VIV10", "VIV14", "VIV16", "VIV25", "VIV26", "VIV27", "VIV28", "VIV29", "VIV31", "VIV32", "VIV33", "VIV34", "VIV38", "VIV39", "VIV40", "VIV41", "VIV42", "VIV81", "VIV37_R", "VIV13_R", "VIV80_R","VIV81_R", "VIV82_R","VIV83_R", "VIV84_R", "HOGAR1", "HOGAR4", "HOGAR7", "HOGAR13", "HOGAR16","HOGAR19", "HOGAR22"]
var_pob = ["POB11_R", "POB17_R", "POB42_R", "POB84_R", "MIG7_R","INDI1_R", "DISC1_R","EDU49_R","SCONY1_R", "SCONY4_R", "SCONY7_R", "RELIG1_R", "RELIG2_R","EDU25", "EDU28", "SALUD1", "SALUD2", "SALUD7", "SALUD8"]
var_ec = ["ECO1_R", "ECO28_R", "ECO34_R"]

In [6]:
#building the demographic database
geo_db = BD_MANZANAS.select(var_viv)
geo_db = geo_db.join(BD_MANZANAS.select(var_pob))
geo_db = geo_db.join(BD_MANZANAS.select(var_ec))

In [7]:
geo_db.createOrReplaceTempView("geo_db")
geo_db.printSchema()

22/11/28 14:47:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
root
 |-- CVEGEO: string (nullable = true)
 |-- VIV0: double (nullable = true)
 |-- VIV1: double (nullable = true)
 |-- VIV7: double (nullable = true)
 |-- VIV8: double (nullable = true)
 |-- VIV9: double (nullable = true)
 |-- VIV10: double (nullable = true)
 |-- VIV14: double (nullable = true)
 |-- VIV16: double (nullable = true)
 |-- VIV25: double (nullable = true)
 |-- VIV26: double (nullable = true)
 |-- VIV27: double (nullable = true)
 |-- VIV28: double (nullable = true)
 |-- VIV29: double (nullable = true)
 |-- VIV31: double (nullable = true)
 |-- VIV32: double (nullable = true)
 |-- VIV33: double (nullable = true)
 |-- VIV34: double (nullable = true)
 |-- VIV38: double (nullable = true)
 |-- VIV39: double (nullable = true)
 |-- VIV40: double (nullable = true)
 |-- VIV41: double (nullable = true)
 |-- 

In [195]:
# Read Nuevo Leon economic units file
BD_DENUE = spark.read.parquet(f"/Users/antoniopatjane/Downloads/19.parquet")
BD_DENUE.createOrReplaceTempView("denue")
BD_DENUE.cache()

DataFrame[id: bigint, clee: string, nom_estab: string, raz_social: string, codigo_act: string, nombre_act: string, per_ocu: string, tipo_vial: string, nom_vial: string, tipo_v_e_1: string, nom_v_e_1: string, tipo_v_e_2: string, nom_v_e_2: string, tipo_v_e_3: string, nom_v_e_3: string, numero_ext: string, letra_ext: string, edificio: string, edificio_e: string, numero_int: string, letra_int: string, tipo_asent: string, nomb_asent: string, tipoCenCom: string, nom_CenCom: string, num_local: string, cod_postal: string, cve_ent: string, entidad: string, cve_mun: string, municipio: string, cve_loc: string, localidad: string, ageb: string, manzana: string, telefono: string, correoelec: string, www: string, tipoUniEco: string, latitud: double, longitud: double, fecha_alta: string, geometry: udt]

In [24]:
#Variables DENUE
denue_var = ["id", "clee", "nom_estab", "codigo_act", "cod_postal","cve_mun", "municipio", "longitud", "latitud", "geometry"]

In [196]:
denue_nl = BD_DENUE.select(denue_var)
denue_nl.createOrReplaceTempView("denue_nl")

In [26]:
BD_DENUE.printSchema()
#https://www.inegi.org.mx/rnm/index.php/catalog/668/variable-groups/VG3

root
 |-- id: long (nullable = true)
 |-- clee: string (nullable = true)
 |-- nom_estab: string (nullable = true)
 |-- raz_social: string (nullable = true)
 |-- codigo_act: string (nullable = true)
 |-- nombre_act: string (nullable = true)
 |-- per_ocu: string (nullable = true)
 |-- tipo_vial: string (nullable = true)
 |-- nom_vial: string (nullable = true)
 |-- tipo_v_e_1: string (nullable = true)
 |-- nom_v_e_1: string (nullable = true)
 |-- tipo_v_e_2: string (nullable = true)
 |-- nom_v_e_2: string (nullable = true)
 |-- tipo_v_e_3: string (nullable = true)
 |-- nom_v_e_3: string (nullable = true)
 |-- numero_ext: string (nullable = true)
 |-- letra_ext: string (nullable = true)
 |-- edificio: string (nullable = true)
 |-- edificio_e: string (nullable = true)
 |-- numero_int: string (nullable = true)
 |-- letra_int: string (nullable = true)
 |-- tipo_asent: string (nullable = true)
 |-- nomb_asent: string (nullable = true)
 |-- tipoCenCom: string (nullable = true)
 |-- nom_CenCom: 

In [186]:
todas_var = ["ECO1_R", "ECO28_R", "ECO34_R", "POB11_R", "POB17_R", "POB42_R", "POB84_R", "MIG7_R","INDI1_R", "DISC1_R","EDU49_R","SCONY1_R", "SCONY4_R", "SCONY7_R", "RELIG1_R", "RELIG2_R","EDU25", 
             "EDU28", "SALUD1", "SALUD2", "SALUD7", "SALUD8","CVEGEO", "VIV0", "VIV1", "VIV7", "VIV8", "VIV9", "VIV10", "VIV14", "VIV16", "VIV25", "VIV26", "VIV27", "VIV28", "VIV29", "VIV31", "VIV32", "VIV33", "VIV34", "VIV38", "VIV39", "VIV40", "VIV41", "VIV42", "VIV81", "VIV37_R", "VIV13_R", "VIV80_R","VIV81_R", "VIV82_R","VIV83_R", "VIV84_R", "HOGAR1", "HOGAR4", "HOGAR7", "HOGAR13", "HOGAR16","HOGAR19", "HOGAR22"]


In [255]:
#Selected Economic Activities key codes
restaurantes = ["722511", "722512", "722513", "722514", "722515"]
negocios = {"NEGOCIO_TIENDITA": "461110", "NEGOCIO_ROPA": "463211", "NEGOCIO_TECNOLOGIA": "466211", "NEGOCIO_RESTAURANTE": "722", "NEGOCIO_PAPELERIA":"465311", "NEGOCIO_GIMNASIO": "713943", "NEGOCIO_ESTETICA": "812110"} 
codigos_negocios = ("461110", "463211", "466211", "722511", "722512", "722513", "722514", "722515", "465311", "713943", "812110")


In [198]:
#build the table with selected economic activities
negocios_total = spark.sql("""SELECT *
                            FROM denue_nl
                            WHERE denue_nl.codigo_act IN {}""".format(codigos_negocios))

In [ ]:
#negocios_total.show(1)
#if you want to visualize an example of the variables in the table

In [29]:
#total count of business' in the database
negocios_total.count()

48154

In [30]:
negocios_total.createOrReplaceTempView("negocios1")

In [31]:
negocios_total.show()

+-------+--------------------+-------------------+----------+----------+-------+--------------------+-------------+-----------+--------------------+
|     id|                clee|          nom_estab|codigo_act|cod_postal|cve_mun|           municipio|     longitud|    latitud|            geometry|
+-------+--------------------+-------------------+----------+----------+-------+--------------------+-------------+-----------+--------------------+
|8731270|19018461110013661...|    ABARROTES CHIKY|    461110|     66003|    018|              García|-100.57530191|25.80513946|POINT (2642201.68...|
|3004671|19021461110017381...|    ABARROTES CHILO|    461110|     66084|    021|    General Escobedo| -100.3853926|25.83100522|POINT (2661124.01...|
|7704463|19039461110080011...|    ABARROTES CHILO|    461110|     64270|    039|           Monterrey|-100.33439389| 25.7249145|POINT (2666348.87...|
|3034921|19039461110018541...|    ABARROTES CHICO|    461110|     64720|    039|           Monterrey|-100.

In [19]:
import firebase_admin

cred = firebase_admin.credentials.Certificate("inegi-497e7-firebase-adminsdk-s4h6a-19e076bdc5.json")
default_app = firebase_admin.initialize_app(cred, {
	'databaseURL': 'https://inegi-497e7-default-rtdb.firebaseio.com/'
	})

In [319]:
from firebase_admin import db

ref_neg = db.reference("/Busqueda1/CodigoNegocio")
neg = ref_neg.get()
neg = neg[0]
print(neg)

NEGOCIO_GIMNASIO


In [320]:
ref_pos = db.reference("/Busqueda1/CP")
pos = ref_pos.get()
pos = pos[0]
print(pos)

64000


In [321]:
#Getting location data
#esta es la parte que se tiene que conectar con el chatbot !!!???

zip_code= pos
act_cod = negocios[neg]

In [322]:
#print nearby similar business info
resultado = spark.sql(""" SELECT *
                        FROM negocios1
                        WHERE negocios1.codigo_act = {} AND negocios1.cod_postal = {} """.format(act_cod, zip_code)) 
business_number = resultado.count()
print(business_number)

13


In [334]:
ref  = db.reference("/Busqueda1")
ref.update({"NumNegocios": business_number})

In [324]:
negocio_count= spark.sql("""SELECT codigo_act, count(codigo_act)
  FROM negocios1 
 WHERE cod_postal  = {}
 GROUP by codigo_act """.format(zip_code))

In [325]:
negocio_count.show()

+----------+-----------------+
|codigo_act|count(codigo_act)|
+----------+-----------------+
|    461110|              120|
|    465311|               38|
|    466211|               44|
|    463211|              662|
|    722514|              253|
|    722513|               82|
|    713943|               13|
|    722515|              108|
|    722511|              278|
|    722512|               47|
|    812110|              269|
+----------+-----------------+



In [326]:
#def invertir_diccionario(diccionario):
 #return{valor:clave for clave, valor in diccionario.items()}

#new_dict = {}


#for org_key in restaurantes:
#	new_dict[org_key] = 'NEGOCIO_RESTAURANTE'


#TempNeg = invertir_diccionario(negocios)
#invNegocios = {**TempNeg,**new_dict}

In [327]:
invNegocios = {'461110': 'NEGOCIO_TIENDITA', '463211': 'NEGOCIO_ROPA', '466211': 'NEGOCIO_TECNOLOGIA', '722': 'NEGOCIO_RESTAURANTE', '465311': 'NEGOCIO_PAPELERIA', '713943': 'NEGOCIO_GIMNASIO', '812110': 'NEGOCIO_ESTETICA', '722511': 'NEGOCIO_RESTAURANTE', '722512': 'NEGOCIO_RESTAURANTE', '722513': 'NEGOCIO_RESTAURANTE', '722514': 'NEGOCIO_RESTAURANTE', '722515': 'NEGOCIO_RESTAURANTE'}

In [328]:
cuentaTotal = negocio_count.toPandas()

In [329]:
cuentaTotal.replace({"codigo_act": invNegocios}, inplace=True)
negoMax = cuentaTotal.max()
negoMin = cuentaTotal.min()


In [330]:
cuentaTotal.head()

,codigo_act,count(codigo_act)
0,NEGOCIO_TIENDITA,120
1,NEGOCIO_PAPELERIA,38
2,NEGOCIO_TECNOLOGIA,44
3,NEGOCIO_ROPA,662
4,NEGOCIO_RESTAURANTE,253


In [331]:
ref  = db.reference("/Busqueda1")

nombrNegocioMax = negoMax[0]
ref.update({"NegociosMas": nombrNegocioMax})
numMax = negoMax[1]
ref.update({"NegociosMasNum": str(numMax)})


nombrNegocioMin = negoMin[0]
ref.update({"NegociosMenos": nombrNegocioMin})
numMin = negoMin[1]
ref.update({"NegociosMenosNum": str(numMin)})

1. buscar las caracteristicas del lugar ya sea con un join de la columna de manzanas entre la denue y la otra oooo con el zip code 
2. unir el automl+geo

In [335]:
import time

time.sleep(40)

ref = db.reference("/")

import json

with open("busquedas.json", "r") as f:
	file_contents = json.load(f)
ref.set(file_contents)

Rests_Nacional = spark.sql("""SELECT 
                                  1 AS klass,
                                  id, clee,
                                  LOWER(nom_estab) nom_estab ,
                                  cve_ent,
                                  per_ocu,
                                  geometry,
                                  cod_postal,
                                  codigo_act
                            FROM denue
                            WHERE denue.codigo_act LIKE '722%' """)#in ('721111','722511', '722512','722513', '722515')""")
count1 = Rests_Nacional.count() #datos en la primera clase
#count1

#Seleccionar el resto de los negocios
otros_Nacional = BD_DENUE.filter(BD_DENUE.codigo_act != "722%")\
                     .select(lit("2").alias("klass"),\
                      col("id"),\
                      col("clee"),\
                      lower(col("nom_estab")).alias("nom_estab"),\
                      col("cve_ent"),\
                      col("per_ocu"),\
                      col("geometry"),\
                      col("cod_postal"),\
                      col("codigo_act"))\
                     .orderBy(rand())\
                     .limit(22057) #ver esto
otros_Nacional.count()

Nacional = Rests_Nacional.union(otros_Nacional).orderBy(rand())
Nacional.count()

Nacional.createOrReplaceTempView("nacional")
Nacional.cache()

DenueGeo = spark.sql("""SELECT *
                            FROM nacional
                            INNER JOIN geo_db ON nacional.clee  =  geo_db.CVEGEO""")
DenueGeo.createOrReplaceTempView("DenueGeo")
DenueGeo.printSchema()

In [333]:
byCP = DenueGeo.groupBy('cod_postal','codigo_act').avg()
byCP.printSchema()

root
 |-- cod_postal: string (nullable = true)
 |-- codigo_act: string (nullable = true)
 |-- avg(id): double (nullable = true)
 |-- avg(VIV0): double (nullable = true)
 |-- avg(VIV1): double (nullable = true)
 |-- avg(VIV7): double (nullable = true)
 |-- avg(VIV8): double (nullable = true)
 |-- avg(VIV9): double (nullable = true)
 |-- avg(VIV10): double (nullable = true)
 |-- avg(VIV14): double (nullable = true)
 |-- avg(VIV16): double (nullable = true)
 |-- avg(VIV25): double (nullable = true)
 |-- avg(VIV26): double (nullable = true)
 |-- avg(VIV27): double (nullable = true)
 |-- avg(VIV28): double (nullable = true)
 |-- avg(VIV29): double (nullable = true)
 |-- avg(VIV31): double (nullable = true)
 |-- avg(VIV32): double (nullable = true)
 |-- avg(VIV33): double (nullable = true)
 |-- avg(VIV34): double (nullable = true)
 |-- avg(VIV38): double (nullable = true)
 |-- avg(VIV39): double (nullable = true)
 |-- avg(VIV40): double (nullable = true)
 |-- avg(VIV41): double (nullable = t